In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
from bokeh.plotting import output_notebook, figure, show
from bokeh.models import ColumnDataSource, Div, Select, Button, ColorBar, CustomJS
from bokeh.layouts import row, column, layout
from bokeh.transform import cumsum, linear_cmap
from bokeh.palettes import Blues8, Spectral3
from bokeh.plotting import figure, output_file, show


output_notebook()

# Visualisation libraries
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import pandas as pd
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual

import folium 
from folium import plugins
plt.style.use("fivethirtyeight")


from plotly.offline import iplot
from plotly import tools
import plotly.graph_objects as go
import plotly.express as px
import plotly.offline as py
import plotly.figure_factory as ff
py.init_notebook_mode(connected=True)

In [ ]:
# Read the file
train_df = pd.read_csv("/kaggle/input/covid19-global-forecasting-week-1/train.csv")
test_df = pd.read_csv("/kaggle/input/covid19-global-forecasting-week-1/test.csv")
submission_csv = pd.read_csv("/kaggle/input/covid19-global-forecasting-week-1/submission.csv")


In [ ]:
train_df.head()

In [ ]:

train_df["Date"] = pd.to_datetime(train_df["Date"])

In [ ]:
#Basic EDA
train_df.describe()

In [ ]:
train_df.info()

In [ ]:
#Getting the summary of Nan's in the dataset
train_df.isnull().sum()

In [ ]:
#filling up null with Country/Region wherever the province/State is NAN
train_df['Province/State']=train_df['Province/State'].fillna(train_df['Country/Region'])

In [ ]:
#correlation Graph

In [ ]:

sns.heatmap(train_df.corr(),cmap='Greens',annot=True);

In [ ]:
# visualize the relationship between the features and the response using scatterplots
sns.pairplot(train_df, 
             x_vars='ConfirmedCases', 
             y_vars='Fatalities', 
             height=7, 
             aspect=1, 
             kind='reg');

In [ ]:
#interactive Countries along with their Fatalities
from ipywidgets import FloatSlider

@interact(x=train_df['Country/Region'].unique(), y=train_df.groupby('Country/Region')['Fatalities'].sum())

def g(x,y):
         return(x,y)

In [ ]:
confirmed_total_date = train_df.groupby(['Date']).agg({'ConfirmedCases':['sum']})
fatalities_total_date = train_df.groupby(['Date']).agg({'Fatalities':['sum']})
total_date = confirmed_total_date.join(fatalities_total_date)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(17,7))
total_date.plot(ax=ax1)
ax1.set_title("Global confirmed cases", size=13)
ax1.set_ylabel("Number of cases", size=13)
ax1.set_xlabel("Date", size=13)
fatalities_total_date.plot(ax=ax2, color='orange')
ax2.set_title("Global deceased cases", size=13)
ax2.set_ylabel("Number of cases", size=13)
ax2.set_xlabel("Date", size=13)

In [ ]:
# The global curve shows a rich fine structure, but these numbers are strongly affected by the vector zero country, China. Given that COVID-19 started there, during the initial expansion of the virus there was no reliable information about the real infected cases. In fact, the criteria to consider infection cases was modified around 2020-02-11, which strongly perturbed the curve as you can see from the figure.

In [ ]:
#Graph which shows how the corona Virus got spread out in the whole world.


temp = train_df.groupby(['Date', 'Country/Region'])['ConfirmedCases'].sum().reset_index()
temp['Date'] = pd.to_datetime(temp['Date'])
temp['Date'] = temp['Date'].dt.strftime('%m/%d/%Y')
temp['size'] = temp['ConfirmedCases'].pow(0.3) * 3.5


fig = px.scatter_geo(temp, locations="Country/Region", locationmode='country names', 
                     color="ConfirmedCases", size='size', hover_name="Country/Region", 
                     range_color=[1,100],
                     projection="natural earth", animation_frame="Date", 
                     title='COVID-19: Cases Over Time', color_continuous_scale="Reds")
fig.show()

In [ ]:
Country=pd.DataFrame()
#temp = train_df.groupby(["Country/Region"])["ConfirmedCases"].sum().reset_index()
temp = train_df.loc[train_df["Date"]==train_df["Date"][len(train_df)-1]].groupby(['Country/Region'])["ConfirmedCases"].sum().reset_index()
Country['Name']=temp["Country/Region"]
Country['Values']=temp["ConfirmedCases"]
#Country['Deaths']=temp["Fatalities"]

fig = px.choropleth(Country, locations='Name',
                    locationmode='country names',
                    color="Values")
fig.update_layout(title="Corona spread on 21-03-2020")
fig.show()

In [ ]:
train_df[train_df['Date']=='2020-03-21']

In [ ]:
#Count of COnfirmed Cases as well as fatalies along with their Country/Regions
temp1 = train_df.groupby(["Country/Region"])["ConfirmedCases","Fatalities"].sum().reset_index()
temp1

In [ ]:
#forecasting the confirmed cases and fatalities by corona

Data = train_df.groupby("Date").sum().reset_index()
Data["Date"]= pd.to_datetime(Data["Date"])
source = ColumnDataSource(Data)
p = figure(x_axis_type='datetime')



p.line(x='Date', y='ConfirmedCases', line_width=2, source=source, legend_label='Confirmed Corona Cases')
p.line(x='Date', y='Fatalities', line_width=2, source=source, color=Spectral3[2], legend_label='Death by Corona')

p.yaxis.axis_label = 'Activity of Corona period of time'
show(p)

In [ ]:
#Observation:
#There are two fluctuation point on the period of Corona one between 2-10-2020 and 2-15-2020 
#and other at 3-10-2020 and 3-15-2020, which is exactly a month period.
#So it might follow a cyclical pattern

In [ ]:
train_df['Country/Region'].unique()

In [ ]:
China_cases=train_df.loc[train_df['Country/Region']=='China'].groupby('Date')['ConfirmedCases'].sum().reset_index()
Italy_cases=train_df.loc[train_df['Country/Region']=='Italy'].groupby('Date')['ConfirmedCases'].sum().reset_index()
Iran_cases=train_df.loc[train_df['Country/Region']=='Iran'].groupby('Date')['ConfirmedCases'].sum().reset_index()
India_cases=train_df.loc[train_df['Country/Region']=='India'].groupby('Date')['ConfirmedCases'].sum().reset_index()
fig=go.Figure()
fig.add_trace(go.Scatter(x=China_cases.Date,y=China_cases['ConfirmedCases'], name="Cases in China",
                         line_color='deepskyblue'))
fig.add_trace(go.Scatter(x=Italy_cases.Date,y=Italy_cases['ConfirmedCases'], name='Cases in Italy'
                        ,line_color='red'))
fig.add_trace(go.Scatter(x=Iran_cases.Date,y=Iran_cases['ConfirmedCases'],name="Cases in Iran",
                        line_color='green'))
fig.add_trace(go.Scatter(x=India_cases.Date,y=India_cases['ConfirmedCases'],name='Cases in India',
                        line_color='Yellow'))

In [ ]:
china_cases=train_df.loc[train_df['Country/Region']=='China'].groupby('Date')['Fatalities'].sum().reset_index()
italy_cases=train_df.loc[train_df['Country/Region']=='Italy'].groupby('Date')['Fatalities'].sum().reset_index()
iran_cases=train_df.loc[train_df['Country/Region']=='Iran'].groupby('Date')['Fatalities'].sum().reset_index()
india_cases=train_df.loc[train_df['Country/Region']=='India'].groupby('Date')['Fatalities'].sum().reset_index()
fig=go.Figure()
fig.add_trace(go.Scatter(x=china_cases.Date,y=china_cases['Fatalities'],name='Death in China',
                        line_color='deepskyblue'))

fig.add_trace(go.Scatter(x=italy_cases.Date,y=italy_cases['Fatalities'],name='Death in italy',
                        line_color='red'))

fig.add_trace(go.Scatter(x=iran_cases.Date,y=iran_cases['Fatalities'],name='Death in iran',
                        line_color='green'))

fig.add_trace(go.Scatter(x=india_cases.Date,y=india_cases['Fatalities'],name='Death in india',
                        line_color='yellow'))

In [ ]:
#as we can see the confirmed cases in China is the largest followed by Italy, Iran and India 
#from the Trend we can see that Italy and Iran is on Verge of following the same Trend as China.

#whereas the fatality rate has been the largest in Italy followed by China, Iran and India.

In [ ]:
#Survival Probability : 
#As the Name Suggest, the survival probability is the proportion of units that survive beyond a specified time.
#The survival function, conventionally denoted by S,is the probability that the event (say, death) has not occurred yet: S(t)=Pr(T>t)

#Hazard Rate:
#The hazard function is the instantaneous rate of failure at a given time.

#The hazard function h(t) is the event (death) rate at time t, conditional on survival until t (i.e., T≥t): h(t)=p(t)/S(t)    

In [ ]:
import math
def Survival(Country):
    Sx = [] 
    d = 1
    Ld = 0
    temp_ = train_df.loc[train_df["Country/Region"]==Country]
    temp = temp_.groupby(['Date'])['Fatalities','ConfirmedCases'].sum().reset_index()
    temp["Survival Probability"] = 0
    temp["Hazard Rate"] = 0
    Hr = []
    for i in range(len(temp)):
        delta = 1
        d = temp["Fatalities"][i]
        n = temp["ConfirmedCases"][i]
        L = Ld + (math.pow((d/n),delta))
        S = math.exp(-L)
        Hr.append(L)
        Sx.append(S)
        d= temp["Fatalities"][i]
        Ld = 0
    temp["Survival Probability"] = Sx
    temp["Hazard Rate"] = Hr
    return temp
    
India_df = Survival("India")


In [ ]:
temp_df = train_df.loc[train_df["Date"]=="2020-03-21"].groupby("Country/Region")["ConfirmedCases","Fatalities"].sum().reset_index()
temp=pd.DataFrame()
temp["Index"] = ["Korea,South","Spain","Iran","Italy","China","Others"]
t = temp_df.sort_values(by="ConfirmedCases").tail()["ConfirmedCases"].values
values = []
for i in range(0,5):
    values.append(t[i])
values.append(sum(temp_df.loc[~temp_df["Country/Region"].isin(temp["Index"])]["ConfirmedCases"]))
temp["Values"]=values

fig = go.Figure(data=[go.Pie(labels=temp["Index"], values=temp["Values"],hole=0.2)])
fig.show()

In [ ]:
#The novel coronavirus that originated in Wuhan, China, in late December.
#The virus is now known to have infected more than 240,000 people around the world and killed more than 9,800 of them.
#The virus has also spread to about 160 countries and regions around the world in less than three months.
#Of these affected countries let's try to understand the ground truth of some of them.
 # * China
 # * Italy
 # * Iran
 # * South Korea

In [ ]:
temp = train_df.loc[(train_df["Country/Region"]=="China") & (train_df["Date"]=="2020-03-20")].groupby(["Province/State","Lat","Long"])["ConfirmedCases"].sum().reset_index()
map = folium.Map(location=[34, 100], zoom_start=3.5,tiles='Stamen Toner')

for lat, lon, value, name in zip(temp['Lat'], temp['Long'], temp['ConfirmedCases'], temp["Province/State"]):
    folium.CircleMarker([lat, lon],
                        radius=value*0.007,
                        popup = ('<strong>State</strong>: ' + str(name).capitalize() + '<br>'
                                '<strong>Active Cases</strong>: ' + str(value) + '<br>'),
                        color='red',
                        
                        fill_color='red',
                        fill_opacity=0.3 ).add_to(map)
map

In [ ]:
temp = train_df.loc[(train_df["Country/Region"]=="China")].groupby(["Date"])["ConfirmedCases"].sum().reset_index()
fig = px.bar(temp, x='Date', y='ConfirmedCases',
             hover_data=['ConfirmedCases'], color='ConfirmedCases',
             labels={'pop':'Total Number of confirmed Cases'}, height=400)
fig.show()

In [ ]:
temp = train_df.loc[(train_df["Country/Region"]=="India") & (train_df["Date"]=="2020-03-20")].groupby(["Province/State","Lat","Long"])["ConfirmedCases"].sum().reset_index()
map = folium.Map(location=[34, 100], zoom_start=3.5,tiles='Stamen Toner')

for lat, lon, value, name in zip(temp['Lat'], temp['Long'], temp['ConfirmedCases'], temp["Province/State"]):
    folium.CircleMarker([lat, lon],
                        radius=value*0.007,
                        popup = ('<strong>State</strong>: ' + str(name).capitalize() + '<br>'
                                '<strong>Active Cases</strong>: ' + str(value) + '<br>'),
                        color='red',
                        
                        fill_color='red',
                        fill_opacity=0.3 ).add_to(map)
map

In [ ]:

from plotly.subplots import make_subplots
import plotly.graph_objects as go

Sk_df = train_df.loc[train_df["Country/Region"]=="Korea, South"].groupby("Date").sum().reset_index()
Iran_df = train_df.loc[train_df["Country/Region"]=="Iran"].groupby("Date").sum().reset_index()

fig = make_subplots(rows=1, cols=2, shared_yaxes=True)

fig.add_trace(go.Bar(x=Sk_df.Date, y=Sk_df.ConfirmedCases, name="South Korea"),1, 1)

fig.add_trace(go.Bar(x=Iran_df.Date, y=Iran_df.ConfirmedCases, name ="Iran"),1, 2)

#fig.update_layout(coloraxis=dict(colorscale='Bluered_r'), showlegend=False)
fig.update_layout(
    title_text="Disease Spread Pattern in South Korea and Iran")
fig.show()


In [ ]:
temp1


In [ ]:
x = train_df[['Lat', 'Long', 'Date']]
y1 = train_df[['ConfirmedCases']]
y2 = train_df[['Fatalities']]
x_test = test_df[['Lat', 'Long', 'Date']]
from sklearn import preprocessing

from sklearn.preprocessing import OneHotEncoder
labelencoder_x=preprocessing.LabelEncoder()
x['Date']=labelencoder_x.fit_transform(x['Date'])

x_test['Date']=labelencoder_x.fit_transform(x_test['Date'])
from sklearn.ensemble import RandomForestRegressor
rf=RandomForestRegressor(n_estimators=100,random_state=42)
rf.fit(x,y1)

In [ ]:
rf.score(x,y1)

In [ ]:
predicted=rf.predict(x_test)

In [ ]:
expected=y_test

In [ ]:
from sklearn.tree import DecisionTreeClassifier
Tree_model = DecisionTreeClassifier(criterion='entropy')
##
Tree_model.fit(x,y1)
pred1 = Tree_model.predict(x_test)
pred1 = pd.DataFrame(pred1)
pred1.columns = ["ConfirmedCases_prediction"]
Tree_model.fit(x,y2)
pred2 = Tree_model.predict(x_test)
pred2 = pd.DataFrame(pred2)
pred2.columns = ["Death_prediction"]

In [ ]:
Sub = pd.read_csv("../input/covid19-global-forecasting-week-1/submission.csv")
Sub.columns
sub_new = Sub[["ForecastId"]]

In [ ]:
OP = pd.concat([pred1,pred2,sub_new],axis=1)
OP.head()
OP.columns = ['ConfirmedCases', 'Fatalities', 'ForecastId']
OP = OP[['ForecastId','ConfirmedCases', 'Fatalities']]

In [ ]:
OP["ConfirmedCases"] = OP["ConfirmedCases"].astype(int)
OP["Fatalities"] = OP["Fatalities"].astype(int)

In [ ]:
OP.head()

In [ ]:
#### By using DecisionTreeRegressor
from sklearn.tree import DecisionTreeRegressor  
regressor = DecisionTreeRegressor(random_state = 0) 
regressor.fit(x,y1)
pred_r1 = regressor.predict(x_test)
pred_r1 = pd.DataFrame(pred_r1)
pred_r1.columns = ["ConfirmedCases_prediction"]
regressor.fit(x,y2)
pred_r2 = regressor.predict(x_test)
pred_r2 = pd.DataFrame(pred_r2)
pred_r2.columns = ["Death_prediction"]
OP_dr = pd.concat([sub_new,pred_r1,pred_r2],axis=1)
OP_dr.head()
OP_dr.columns = [ 'ForecastId','ConfirmedCases', 'Fatalities']
OP_dr["ConfirmedCases"] = OP_dr["ConfirmedCases"].astype(int)
OP_dr["Fatalities"] = OP_dr["Fatalities"].astype(int)
OP_dr.head()

In [ ]:

## Random Forest regressor
from sklearn.ensemble import RandomForestRegressor
rand_reg = RandomForestRegressor()
rand_reg.fit(x,y1)
pred_ra1 = rand_reg.predict(x_test)
pred_ra1 = pd.DataFrame(pred_ra1)
pred_ra1.columns = ["ConfirmedCases_prediction"]
rand_reg.fit(x,y2)
pred_ra2 = rand_reg.predict(x_test)
pred_ra2 = pd.DataFrame(pred_ra2)
pred_ra2.columns = ["Death_prediction"]
OP_ra = pd.concat([sub_new,pred_ra1,pred_ra2],axis=1)
OP_ra.head()
OP_ra.columns = [ 'ForecastId','ConfirmedCases', 'Fatalities']
OP_ra["ConfirmedCases"] = OP_ra["ConfirmedCases"].astype(int)
OP_ra["Fatalities"] = OP_ra["Fatalities"].astype(int)
OP_ra.head()

In [ ]:
OP_ra.to_csv("submission.csv",index=False)